In [73]:
#importing necessary packages
!pip install BeautifulSoup4
!pip install lxml 
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [74]:
#fetching data from the url
url = 'https://www.medicineindia.org/hospitals-in-state/tamilnadu'
source = requests.get(url).text

In [75]:
#object of beautifulsoup
soup = BeautifulSoup(source, 'html')

In [76]:
#finding the table
table=soup.find('table')

In [77]:
#Create dataframe of the required attributes
column_names = ['Hospital name','State','City']
df = pd.DataFrame(columns = column_names)

In [78]:
#scraping the web content for tabular data
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

In [79]:
df.head()

,Hospital name,State,City
0,A A Hospital,Tamilnadu,Hosur
1,A C Hospital,Tamilnadu,Chennai
2,A.C. Hospital,Tamilnadu,Salem
3,A.G. Eye Care Hospitals (U Nit Of Dr. A.G. Eye...,Tamilnadu,Chennai
4,A.G. Hospital,Tamilnadu,Coimbatore


In [80]:
df.shape

(918, 3)

In [81]:
#our required dataframe
df.drop(['State'],axis=1)

,Hospital name,City
0,A A Hospital,Hosur
1,A C Hospital,Chennai
2,A.C. Hospital,Salem
3,A.G. Eye Care Hospitals (U Nit Of Dr. A.G. Eye...,Chennai
4,A.G. Hospital,Coimbatore
...,...,...
913,Vn Neuro Hospital & Vn Neuro Centre,Madurai
914,West Madras Medical Centre,Chennai
915,Womens And Children Hospital,Chennai
916,Xcellent Care Super Speciality Hospital (P) Ltd,Chennai


In [82]:
import numpy as np
df_chennai = df[df['City'] == 'Chennai']
df_chennai

,Hospital name,State,City
1,A C Hospital,Tamilnadu,Chennai
3,A.G. Eye Care Hospitals (U Nit Of Dr. A.G. Eye...,Tamilnadu,Chennai
6,A.N.N Hospital,Tamilnadu,Chennai
8,A.V. Hospitals,Tamilnadu,Chennai
10,Aakash Hospital,Tamilnadu,Chennai
...,...,...,...
907,Vinayagam Laser Hospital,Tamilnadu,Chennai
914,West Madras Medical Centre,Tamilnadu,Chennai
915,Womens And Children Hospital,Tamilnadu,Chennai
916,Xcellent Care Super Speciality Hospital (P) Ltd,Tamilnadu,Chennai


In [83]:
! pip install XlsxWriter
df_chennai.to_excel('capstone-input.xlsx',engine='xlsxwriter')

     |████████████████████████████████| 143kB 15.4MB/s eta 0:00:01


In [84]:
df = pd.read_excel('capstone-input.xlsx')

In [85]:
df = df[['Hospital name','City']]

In [86]:
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge geocoder --yes
import geocoder # to get coordinates

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



In [87]:
# define a function to get coordinates of the hospitals
def get_latlng(hospitalname):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Chennai, Tamil Nadu'.format(hospitalname))
        lat_lng_coords = g.latlng
        print(lat_lng_coords)
    return lat_lng_coords
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(hospitalname) for hospitalname in df['Hospital name'].tolist() ]

[13.09791000000007, 80.23021000000006]
[12.926550000000077, 80.11206000000004]
[13.040790000000072, 80.17847000000006]
[13.117090000000076, 80.28653000000008]
[13.159440000000075, 80.30230000000006]
[13.03815000000003, 80.16387000000003]
[13.080830000000049, 80.24852000000004]
[12.99655000000007, 80.26028000000008]
[12.949150000000031, 80.13811000000004]
[13.07430000000005, 80.22033000000005]
[13.06257000000005, 80.23069000000004]
[13.03305000000006, 80.21605000000005]
[13.083620000000053, 80.28252000000003]
[13.163680000000056, 80.25813000000005]
[13.037380000000041, 80.26295000000005]
[13.031250000000057, 80.22290000000004]
[13.077930000000038, 80.24592000000007]
[13.129020000000025, 80.29088000000007]
[13.057060000000035, 80.25517000000008]
[13.033930000000055, 80.24514000000005]
[13.083620000000053, 80.28252000000003]
[12.969920000000059, 80.15001000000007]
[13.037370000000067, 80.19112000000007]
[13.04454000000004, 80.25620000000004]
[13.050770000000057, 80.23463000000004]
[13.044

In [88]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
# merge the coordinates into the original dataframe
df['Latitude'] = df_coords['Latitude']
df['Longitude'] = df_coords['Longitude']

In [89]:
df.to_csv('Hospitallist')

In [90]:
import folium
# get the coordinates of Chennai
address = 'Chennai, Tamil Nadu'

geolocator = Nominatim(user_agent="Chennai Pullingo")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Chennai, Tamil Nadu {}, {}.'.format(latitude, longitude))
# create map of Chennai using latitude and longitude values
map_ch = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, hospitalname in zip(df['Latitude'], df['Longitude'], df['Hospital name']):
    label = '{}'.format(hospitalname)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_ch)  
#hospitals in chennai    
map_ch

The geograpical coordinate of Chennai, Tamil Nadu 13.0801721, 80.2838331.


In [91]:
CLIENT_ID = 'QYP551TAUUEMULHL5ENPALVKC4V0AUY4KXXJHMJ33SIEYF5U' # your Foursquare ID
CLIENT_SECRET = '1YMOHKYD0K4CRGYLK2U5C31DO4H5ETZWL11KLCNAJL3P4MGY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('My foursquare credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My foursquare credentails:
CLIENT_ID: QYP551TAUUEMULHL5ENPALVKC4V0AUY4KXXJHMJ33SIEYF5U
CLIENT_SECRET:1YMOHKYD0K4CRGYLK2U5C31DO4H5ETZWL11KLCNAJL3P4MGY


In [92]:
radius = 1000
LIMIT = 100

venues = []

for lat, long, hospitalname in zip(df['Latitude'], df['Longitude'], df['Hospital name']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()['response']['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            hospitalname,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [93]:
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Hospital name', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(7945, 7)


,Hospital name,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,A C Hospital,13.09791,80.23021,Domino's Pizza,13.099554,80.230880,Pizza Place
1,A C Hospital,13.09791,80.23021,Shri Gopikrishna Theatre,13.095079,80.227423,Indie Movie Theater
2,A C Hospital,13.09791,80.23021,Nilgiris,13.097601,80.237416,Department Store
3,A C Hospital,13.09791,80.23021,Gandhi Nagar Club,13.091507,80.224842,Bar
4,A C Hospital,13.09791,80.23021,Sakthi Pazhamuthir Solai,13.092207,80.223709,Farmers Market


In [94]:
venues_df.groupby(["Hospital name"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Hospital name,,,,,,
A C Hospital,6,6,6,6,6,6
A.G. Eye Care Hospitals (U Nit Of Dr. A.G. Eye Hospitals Private Limited,7,7,7,7,7,7
A.N.N Hospital,19,19,19,19,19,19
Aakash Hospital,6,6,6,6,6,6
Abhijay Hospital (P) Ltd.,17,17,17,17,17,17
...,...,...,...,...,...,...
Vinayagam Laser Hospital,13,13,13,13,13,13
West Madras Medical Centre,69,69,69,69,69,69
Womens And Children Hospital,43,43,43,43,43,43


In [95]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 209 uniques categories.


In [96]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Pizza Place', 'Indie Movie Theater', 'Department Store', 'Bar',
       'Farmers Market', 'Restaurant', 'Food', 'Indian Restaurant',
       'Platform', 'Bus Station', 'Fast Food Restaurant',
       'Ice Cream Shop', 'Asian Restaurant', 'Sandwich Place',
       'Vegetarian / Vegan Restaurant', 'Café', 'Bakery',
       'Clothing Store', 'Tea Room', 'Breakfast Spot',
       'Health & Beauty Service', 'Historic Site', 'Beach',
       'Train Station', 'Hotel', 'Shopping Mall', 'Gas Station', 'Bank',
       'Buffet', 'Italian Restaurant', 'Snack Place',
       'Frozen Yogurt Shop', 'Juice Bar', 'South Indian Restaurant',
       'Multiplex', 'Diner', 'Coffee Shop', "Women's Store",
       'Middle Eastern Restaurant', 'Bistro', 'Fruit & Vegetable Store',
       'Dessert Shop', 'Chinese Restaurant', 'Performing Arts Venue',
       'Gym', 'BBQ Joint', 'Hotel Bar', 'Arcade', 'Food Court',
       'Electronics Store'], dtype=object)

In [97]:
# one hot encoding
hospitals_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
hospitals_onehot['Hospital name'] = venues_df['Hospital name'] 

# move neighborhood column to the first column
fixed_columns = [hospitals_onehot.columns[-1]] + list(hospitals_onehot.columns[:-1])
hospitals_onehot = hospitals_onehot[fixed_columns]

print(hospitals_onehot.shape)
hospitals_onehot.head(5)

(7945, 210)


,Hospital name,ATM,Accessories Store,Adult Boutique,Advertising Agency,Afghan Restaurant,African Restaurant,Airport,Airport Terminal,American Restaurant,...,Theater,Theme Park,Toy / Game Store,Trail,Train,Train Station,Vegetarian / Vegan Restaurant,Video Store,Women's Store,Yoga Studio
0,A C Hospital,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,A C Hospital,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,A C Hospital,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,A C Hospital,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,A C Hospital,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
hospitals_grouped = hospitals_onehot.groupby(["Hospital name"]).mean().reset_index()

hospitals_grouped

,Hospital name,ATM,Accessories Store,Adult Boutique,Advertising Agency,Afghan Restaurant,African Restaurant,Airport,Airport Terminal,American Restaurant,...,Theater,Theme Park,Toy / Game Store,Trail,Train,Train Station,Vegetarian / Vegan Restaurant,Video Store,Women's Store,Yoga Studio
0,A C Hospital,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
1,A.G. Eye Care Hospitals (U Nit Of Dr. A.G. Eye...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
2,A.N.N Hospital,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.105263,0.0,0.0,0.0
3,Aakash Hospital,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.166667,0.000000,0.0,0.0,0.0
4,Abhijay Hospital (P) Ltd.,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324,Vinayagam Laser Hospital,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.076923,0.000000,0.0,0.0,0.0
325,West Madras Medical Centre,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.014493,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.057971,0.0,0.0,0.0
326,Womens And Children Hospital,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.023256,0.0,0.0,0.0,0.0,0.023256,0.000000,0.0,0.0,0.0
327,Xcellent Care Super Speciality Hospital (P) Ltd,0.0,0.029412,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.000000,0.058824,0.0,0.0,0.0


In [98]:
num_top_venues = 5

for hood in hospitals_grouped['Hospital name']:
    print("----"+hood+"----")
    temp = hospitals_grouped[hospitals_grouped['Hospital name'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


----A C Hospital----
                 venue  freq
0  Indie Movie Theater  0.17
1                  Bar  0.17
2     Department Store  0.17
3       Farmers Market  0.17
4           Restaurant  0.17


----A.G. Eye Care Hospitals (U Nit Of Dr. A.G. Eye Hospitals Private Limited----
                 venue  freq
0    Indian Restaurant  0.29
1  Indie Movie Theater  0.14
2             Platform  0.14
3                 Food  0.14
4          Bus Station  0.14


----A.N.N Hospital----
                           venue  freq
0              Indian Restaurant  0.16
1                 Clothing Store  0.11
2           Fast Food Restaurant  0.11
3  Vegetarian / Vegan Restaurant  0.11
4                         Bakery  0.05


----Aakash Hospital----
                 venue  freq
0  Indie Movie Theater  0.17
1        Train Station  0.17
2                Beach  0.17
3          Bus Station  0.17
4        Historic Site  0.17


----Abhijay Hospital (P) Ltd.----
               venue  freq
0   Asian Restaurant  0.18

In [53]:
import numpy as np
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Hospital name']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Hospitals_sorted = pd.DataFrame(columns=columns)
Hospitals_sorted['Hospital name'] = hospitals_grouped['Hospital name']

for ind in np.arange(hospitals_grouped.shape[0]):
    Hospitals_sorted.iloc[ind, 1:] = return_most_common_venues(hospitals_grouped.iloc[ind, :], num_top_venues)

Hospitals_sorted

,Hospital name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,A C Hospital,Indie Movie Theater,Bar,Department Store,Restaurant,Farmers Market,Pizza Place,Hostel,Hospital,Food & Drink Shop,Food
1,A.G. Eye Care Hospitals (U Nit Of Dr. A.G. Eye...,Indian Restaurant,Indie Movie Theater,Bus Station,Food,Pizza Place,Platform,African Restaurant,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint
2,A.N.N Hospital,Indian Restaurant,Vegetarian / Vegan Restaurant,Fast Food Restaurant,Clothing Store,Asian Restaurant,Health & Beauty Service,Department Store,Ice Cream Shop,Café,Breakfast Spot
3,Aakash Hospital,Indie Movie Theater,Beach,Historic Site,Bus Station,Indian Restaurant,Train Station,Advertising Agency,Fish Market,Fruit & Vegetable Store,Frozen Yogurt Shop
4,Abhijay Hospital (P) Ltd.,Asian Restaurant,Indian Restaurant,Bakery,Hotel,Gas Station,Department Store,Café,Restaurant,Buffet,Sandwich Place
...,...,...,...,...,...,...,...,...,...,...,...
324,Vinayagam Laser Hospital,Indian Restaurant,Hotel,Cupcake Shop,Train Station,Department Store,Farmers Market,Fried Chicken Joint,Food Truck,Food Court,Food & Drink Shop
325,West Madras Medical Centre,Indian Restaurant,Vegetarian / Vegan Restaurant,Bakery,Chinese Restaurant,Fast Food Restaurant,Department Store,Restaurant,Café,Clothing Store,Shoe Store
326,Womens And Children Hospital,Indian Restaurant,Café,Hotel,Pizza Place,Seafood Restaurant,Fast Food Restaurant,Farmers Market,Bakery,Korean Restaurant,Bar
327,Xcellent Care Super Speciality Hospital (P) Ltd,Indian Restaurant,Fast Food Restaurant,Restaurant,Pizza Place,Ice Cream Shop,Chinese Restaurant,Vegetarian / Vegan Restaurant,Multiplex,Bar,Kerala Restaurant


In [99]:
# set number of clusters
kclusters = 5

hospitals_clustering = hospitals_grouped.drop(["Hospital name"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(hospitals_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]
kmeans.labels_

array([3, 2, 1, 4, 1, 2, 1, 2, 3, 1, 1, 2, 3, 1, 2, 2, 0, 1, 1, 2, 4, 1,
       1, 1, 1, 2, 1, 1, 1, 4, 4, 2, 1, 1, 2, 2, 4, 1, 3, 3, 1, 3, 0, 1,
       4, 2, 1, 1, 2, 1, 3, 1, 1, 3, 3, 2, 4, 1, 1, 4, 1, 1, 1, 1, 2, 2,
       1, 4, 2, 1, 1, 3, 1, 1, 1, 3, 4, 3, 3, 2, 2, 1, 2, 4, 2, 2, 0, 1,
       1, 2, 1, 4, 1, 1, 3, 4, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 0, 1, 2, 0,
       2, 2, 2, 3, 3, 3, 3, 3, 1, 2, 1, 2, 1, 2, 3, 1, 1, 3, 1, 2, 3, 2,
       2, 4, 1, 1, 1, 3, 2, 3, 0, 3, 1, 2, 1, 3, 1, 1, 2, 3, 1, 3, 3, 1,
       4, 3, 3, 3, 2, 2, 1, 1, 1, 3, 4, 4, 2, 2, 2, 1, 1, 2, 1, 1, 2, 1,
       3, 1, 4, 1, 2, 2, 0, 1, 4, 1, 1, 3, 1, 4, 1, 1, 1, 1, 1, 2, 1, 4,
       1, 1, 3, 3, 2, 1, 2, 1, 4, 1, 3, 1, 2, 3, 1, 1, 1, 1, 1, 2, 2, 1,
       1, 1, 2, 1, 4, 1, 1, 3, 1, 2, 2, 2, 3, 3, 3, 3, 3, 1, 3, 1, 4, 1,
       1, 3, 1, 3, 1, 1, 1, 1, 1, 3, 3, 1, 1, 1, 3, 3, 2, 1, 1, 3, 1, 1,
       3, 2, 1, 1, 3, 1, 2, 1, 1, 1, 3, 3, 1, 1, 2, 1, 1, 1, 4, 1, 2, 1,
       1, 3, 3, 1, 1, 1, 1, 1, 3, 3, 1, 1, 3, 3, 1,

In [100]:
# add clustering labels
#Hospitals_sorted.insert(0, 'Cluster Labels', kmeans.labels)

#ch_merged = ch_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
ch_merged = df.join(Hospitals_sorted.set_index('Hospital name'), on='Hospital name')

ch = ch_merged.dropna()

In [101]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ch['Latitude'], ch['Longitude'], ch['Hospital name'], ch['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [102]:
#5clusters showing distribution of hospitals in chennai 